# Telecom Churn Case Study

## Objectives of the case study¶
Creating a model which can easily find the customers with high probability of churn.
Since prepaid is the kind of service that is used by most of indians, more focused about that.
Finding out the customers with high probability of churn.
Since 80% of the income earned by the top 20% of high value customers.
Finding out the key variable of the customers who have high probability of churn.

In [ ]:
# Let's import the required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#importing the required things from sklearb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

from numpy import loadtxt
from numpy import sort
from sklearn.feature_selection import SelectFromModel

# Warnings Suppressed
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing and installig the xgboost
import sys
!{sys.executable} -m pip install xgboost

from xgboost import XGBClassifier
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
# Import the data set
telecom_churn_df = pd.read_csv(telecom_churn_data.csv")
telecom_churn_df.head()

In [ ]:
# Lets find the description of the given data set
telecom_churn_df.describe

In [ ]:
telecom_churn_df.shape

In [ ]:
## Sorting out the columns with high collinearity

cor1 = telecom_churn_df.corr()
for i in cor1.index:
    for j in cor1.columns:
        if ((i != j) and ((cor1.loc[i,j] > 0.65) or (cor1.loc[i,j] < -0.65))):
            print('Correlation between',i,'&',j,'is  ',cor1.loc[i,j])

In [ ]:
# Lets check for the duplicates in the data set
sum(telecom_churn_df.duplicated(subset = 'mobile_number')) == 0

In [ ]:
# Finding the sum of all missing values
Number_Of_Missing_Values = telecom_churn_df.isnull().sum()

In [ ]:
# Finding the sum of missing values percentage 
Percentage_Of_Missing_Values = 100 * telecom_churn_df.isnull().sum() / len(telecom_churn_df)

In [ ]:
# Concatinating the two tables
Dataframe_Missing_Value = pd.concat([Number_Of_Missing_Values, Percentage_Of_Missing_Values], axis=1)

# Renameing the column names
Columns_Missing_Values = Dataframe_Missing_Value.rename(columns={0: 'Number of Missing Values', 1: '% of Total Missing Values'})

# Display the values in sorting order
Columns_Missing_Values = Columns_Missing_Values[Columns_Missing_Values.iloc[:, 1] != 0].sort_values('% of Total Missing Values', ascending=False).round(2)
print("Total number of missing columns are: " + str(Columns_Missing_Values.shape[0]))
print(Columns_Missing_Values.head(100))


In [ ]:
# Removing the unnecessary variables
telecom_churn_df = telecom_churn_df.drop(['mobile_number','circle_id','last_date_of_month_6','date_of_last_rech_6','date_of_last_rech_data_6',
                                          'last_date_of_month_7','date_of_last_rech_7','date_of_last_rech_data_7',
                                          'last_date_of_month_8','date_of_last_rech_8','date_of_last_rech_data_8'],axis=1)

In [ ]:
# Assigning the value "0" to the nan values
# telecom_churn_df.av_rech_amt_data_8.fillna(value=0, inplace=True)
# telecom_churn_df.av_rech_amt_data_7.fillna(value=0, inplace=True)
# telecom_churn_df.av_rech_amt_data_6.fillna(value=0, inplace=True)
# telecom_churn_df.total_rech_data_8.fillna(value=0, inplace=True)
# telecom_churn_df.total_rech_data_7.fillna(value=0, inplace=True)
# telecom_churn_df.total_rech_data_6.fillna(value=0, inplace=True)

In [ ]:
# Handling the Categorical variables by assigning them dummy's
# Creating dummies for fb_user
# dummy_fb_user_8 = pd.get_dummies(telecom_churn_df['fb_user_8'],drop_first=True)
# dummy_fb_user_7 = pd.get_dummies(telecom_churn_df['fb_user_7'],drop_first=True)
# dummy_fb_user_6 = pd.get_dummies(telecom_churn_df['fb_user_6'],drop_first=True)


In [ ]:
# Concatinating the dummys with the original data set
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_fb_user_8],axis=1)
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_fb_user_7],axis=1)
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_fb_user_6],axis=1)

In [ ]:
# Creating dummies for night pack users
# dummy_night_pck_user_8 = pd.get_dummies(telecom_churn_df['night_pck_user_8'],drop_first=True)
# dummy_night_pck_user_7 = pd.get_dummies(telecom_churn_df['night_pck_user_7'],drop_first=True)
# dummy_night_pck_user_6 = pd.get_dummies(telecom_churn_df['night_pck_user_6'],drop_first=True)

In [ ]:
# Concatinating the dummys with the original data set
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_night_pck_user_8],axis=1)
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_night_pck_user_7],axis=1)
# telecom_churn_df = pd.concat([telecom_churn_df,dummy_night_pck_user_6],axis=1)

In [ ]:
# Let's drop the varibles for which we have created the dummies
# telecom_churn_df = telecom_churn_df.drop(['fb_user_8', 'fb_user_7', 'fb_user_6', 'night_pck_user_8', 'night_pck_user_7', 'night_pck_user_6'], 1)

In [ ]:
# Finding the shape of the data
telecom_churn_df.shape

In [ ]:
# Getting the basic stats about the dataset
telecom_churn_df.describe()

In [ ]:
# Making a variable named "Churn"
telecom_churn_df['Churn'] = np.where( (telecom_churn_df['vol_2g_mb_9']==0 ) & (telecom_churn_df['total_ic_mou_9']==0) & (telecom_churn_df['total_og_mou_9']==0) & (telecom_churn_df['vol_3g_mb_9']==0), 1 ,0 )

In [ ]:
telecom_churn_df.shape

In [ ]:
telecom_churn_df.reset_index(inplace=True)

In [ ]:
# Finding the total rechange amount
telecom_churn_df['Sum_of_Recharge_amount'] = telecom_churn_df['total_rech_amt_7'] + telecom_churn_df['total_rech_amt_6']

In [ ]:
# Let's find the average recharge amount
telecom_churn_df['Average_Recharge_amount'] = telecom_churn_df['Sum_of_Recharge_amount']/2

In [ ]:
Q70 = telecom_churn_df['Average_Recharge_amount'].quantile(0.70)

In [ ]:
telecom_churn_df.drop(telecom_churn_df[telecom_churn_df['Average_Recharge_amount'] <= Q70].index, inplace = True)

In [ ]:
telecom_churn_df.shape

In [ ]:
# Dropping all the columns with prefix "_9"
drop_9_columns = [usg for usg in list(telecom_churn_df) if usg.endswith("_9")]
telecom_churn_df.drop(drop_9_columns, axis=1, inplace=True)

In [ ]:
telecom_churn_df.info()

In [ ]:
telecom_churn_df.fillna(0, inplace=True)

In [ ]:
telecom_churn_df.describe()

In [ ]:
#Sum of missing values
Numberofmissingvalues = telecom_churn_df.isnull().sum()

In [ ]:
# Defining the X and y variables
X=telecom_churn_df.drop('Churn', axis =1)
y=telecom_churn_df.Churn
y.value_counts()

In [ ]:
# Plottingt the graph of ADR  vs Non- ADR
f,ax = plt.subplots(1,2,figsize=(20,10))

colors = ["Green", "Blue"]
labels = 'Non-Churned','Churned'
plt.suptitle('Information on Churn vs NON-Churn labels')
# Plotting the pie chart
y.value_counts().plot.pie(explode=[0,0.05], autopct='%1.2f%%', ax =ax[0], shadow = True,
                                    colors = colors, labels = labels, fontsize =12, startangle=25)
ax[0].set_ylabel('% of ADR vs Non-ADR')

palette = ["Red", "Blue"]
# Plotting the Bar chart
sns.countplot(x='Churn', data=telecom_churn_df)
ax[1].set_xticklabels(['Non-Churn','Churn'],rotation=0, rotation_mode="anchor")
plt.show()

In [ ]:
# Defining the variable "X_train","X_test","y_train","y_test"
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7, random_state = 42)

In [ ]:
#printing the shape of  X_train
print(X_train.shape)

In [ ]:
#printing the shape of  X_test
print(X_test.shape)

In [ ]:
P_C_A = PCA(svd_solver='randomized', random_state= 60)

In [ ]:
P_C_A.fit(X_train)

In [ ]:
len(P_C_A.components_)

In [ ]:
Column_names = list(X_train.columns)
P_C_A_dataframe = pd.DataFrame({'First_Component':P_C_A.components_[0],'Second_Component':P_C_A.components_[1], 'Selected_Feature':Column_names})
P_C_A_dataframe.head()

In [ ]:
#Plotting the principal components

%matplotlib inline
fig = plt.figure(figsize = (10,20))
plt.scatter(P_C_A_dataframe.First_Component, P_C_A_dataframe.Second_Component)
plt.xlabel('First principal component')
plt.ylabel('Second principal component')
plt.title('Plot for principal components')
plt.grid()

for i, txt in enumerate(P_C_A_dataframe.Selected_Feature):
    #Writinig down the annotation
    plt.annotate(txt, (P_C_A_dataframe.First_Component[i],P_C_A_dataframe.Second_Component[i]))
plt.tight_layout()
plt.show()

In [ ]:
# Explainaed variance analysis 
P_C_A.explained_variance_ratio_

plt.figure(figsize=(10,10))
plt.plot(np.cumsum(P_C_A.explained_variance_ratio_),color='red')

plt.grid()
plt.show()

In [ ]:
from sklearn.decomposition import IncrementalPCA
Final_Principal_Components = IncrementalPCA(n_components = 10)
Prinicpal_component_dataframe = Final_Principal_Components.fit_transform(X)
Prinicpal_component_dataframe.shape

In [ ]:
Prinicpal_component_dataframe = pd.DataFrame(Prinicpal_component_dataframe)
Prinicpal_component_dataframe.head()

In [ ]:
# Creating the correlation matrix for Principal component dataframe
Correlation = np.corrcoef(Prinicpal_component_dataframe.transpose())
Correlation

In [ ]:
# Finding the correlation between the variables
plt.figure(figsize = (15,10))
sns.heatmap(Correlation,annot=True,linewidths=.5)

In [ ]:
# Getting the train component and checking its shape
train_Component = Final_Principal_Components.fit_transform(X_train)
train_Component.shape

In [ ]:
# Getting the test component and checking its shape
test_Component = Final_Principal_Components.transform(X_test)
test_Component.shape

In [ ]:
# Let't make the predictions based on the model
def get_score(model,X_train,X_test):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)

In [ ]:
# Using the logistic regression.
Logistic_Classification = LogisticRegression(random_state=42, class_weight='balanced')

In [ ]:
get_score(Logistic_Classification,X_train,X_test)

In [ ]:
get_score(Logistic_Classification,train_Component,test_Component)

In [ ]:
# Let's use the the random forest classification to cl;aasify the variables
Random_Forest_Classification = RandomForestClassifier(random_state = 42, n_estimators= 100,n_jobs =-1, class_weight = {0:1,1:10})
get_score(Random_Forest_Classification,X_train,X_test)

In [ ]:
parameters = { 'n_estimators':[150],'max_features': ['auto','log2','sqrt'] ,'class_weight':['balanced','balanced_subsample'],
          'max_depth' : [2,4,8],'min_samples_leaf':[20,22,24],'criterion' : ['gini']}
Grid_search = GridSearchCV(estimator = RandomForestClassifier(random_state = 42,n_jobs = -1 ), scoring = 'recall', cv =3,
                                                           param_grid = parameters)
Grid_search.fit(X_train,y_train)

In [ ]:
ABC = Grid_search.best_params_
print(ABC)

In [ ]:
Grid_search.best_score_

In [ ]:
Parameter_test = {'min_child_weight':[5,6],'max_depth': range(3,5),'gamma':[0,0.2,0.4]}
gridsearch = GridSearchCV(estimator = XGBClassifier(), param_grid = Parameter_test, scoring='recall',n_jobs=4,iid=False, cv=3)
gridsearch.fit(X_train, y_train)

In [ ]:
# Using the xg boost for boosting the model
XGModel = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=4, min_child_weight=7, gamma=0.4,nthread=4, 
                        subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic',scale_pos_weight=3,seed=29)
XGModel.fit(X_train, y_train)
y_xgboost = XGModel.predict(X_test)

In [ ]:
#Lets find whether the model is overfitted or not
from sklearn.metrics import precision_score, recall_score,f1_score

In [ ]:
# Printing the confusion matrix
print(confusion_matrix(y_test, y_xgboost))


In [ ]:
# Lets predict the value
predictions = [value for value in y_xgboost]

In [ ]:
#Let's check the accuracy of model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))

In [ ]:
#Let's check the precision score of model
precision = precision_score(y_test, predictions)
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))

In [ ]:
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# printing the recall score 
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))


In [ ]:
#Printing the roc for training set
print("roc_auc training set", roc_auc_score(y_train, XGModel.predict_proba(X_train)[:,1]))

In [ ]:
# Printing the roc for test set
print("roc_auc test set", roc_auc_score(y_test, XGModel.predict_proba(X_test)[:,1]))


In [ ]:
# Plotting the graph based on, feature inmportance on the basis of gain.
fig, ax = plt.subplots(figsize=(12,18))
plot_importance(XGModel,importance_type='gain', max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
# We have use the threshold to select the model.
selection = SelectFromModel(XGModel, threshold=0.06, prefit=True)
select_X_train = selection.transform(X_train)

In [ ]:
# training model
selection_model = XGBClassifier()
selection_model.fit(select_X_train, y_train)

In [ ]:
# Testing model
select_X_test = selection.transform(X_test)
y_pred = selection_model.predict(select_X_test)
accuracy_score(y_test,y_pred)